## 2개의 층
### 케라스 API를 통해 패션 MNIST 데이터셋 불러오기

In [4]:
from tensorflow import keras
(train_input, train_target),(test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

이미지 픽셀 값 : 0~1 사이로 변환, 28x28 크기의 2차원 배열을 1차원 배열로 펼친다.

In [5]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)


인공신경망 모델에 층 2개 추가해보기
- 입력층과 출력층 사이에 밀집층 추가
    - 은닉층

> 은닉층: 시그모이드 활성화 함수 사용 / 첫번째 층은 input_shape 로 입력의 크기 지정 필수<br>
> 출력층: 소프트맥스 함수 사용

In [7]:
dense1= keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax') # 10개의 클래스 분류하므로 뉴런 수 10

### 심층 신경망 만들기
- 출력층 가장 마지막에 두기

In [8]:
model = keras.Sequential([dense1, dense2])

In [10]:
model.summary() # 유용한 정보

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 층을 추가하는 다른 방법
추가되는 층을 한번에 알아보는 장점  

In [11]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,),
                       name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

In [12]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Sequential 객체 생성 후 add() 메서드 호출하여 층을 추가해보기

In [17]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               78500     
                                                                 
 dense_8 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 모델 훈련해보기
- 결과 값: 추가되는 층이 성능을 향상시킨다.

In [19]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 1s 841us/step - loss: 0.5647 - accuracy: 0.8078
Epoch 2/5
1500/1500 [==============================] - 1s 796us/step - loss: 0.4091 - accuracy: 0.8528
Epoch 3/5
1500/1500 [==============================] - 1s 784us/step - loss: 0.3748 - accuracy: 0.8647
Epoch 4/5
1500/1500 [==============================] - 1s 781us/step - loss: 0.3524 - accuracy: 0.8730
Epoch 5/5
1500/1500 [==============================] - 1s 803us/step - loss: 0.3351 - accuracy: 0.8798


## 렐루 함수
### 이미지 분류 문제에서 높은 성능을 낼 수 있는 활성화 함수
- 입력이 양수일 경우: 활성화 함수가 없는 것처럼 입력 통과
- 입력이 음수일 경우: 0으로 만든다.

Flatten: 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할
- 입력층과 은닉층 사이에 추가하기 때문에 층이라고 부른다.

In [20]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

flatten 을 추가하면 입력값의 차원을 짐작할 수 있다.

In [21]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 100)               78500     
                                                                 
 dense_10 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


reshape 를 사용하지 않고 모델 훈련해보기
- 시그모이드 함수 사용보다 성능이 향상됨

In [24]:
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [25]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 1s 871us/step - loss: 0.5258 - accuracy: 0.8150
Epoch 2/5
1500/1500 [==============================] - 1s 836us/step - loss: 0.3919 - accuracy: 0.8597
Epoch 3/5
1500/1500 [==============================] - 1s 815us/step - loss: 0.3577 - accuracy: 0.8726
Epoch 4/5
1500/1500 [==============================] - 1s 807us/step - loss: 0.3353 - accuracy: 0.8799
Epoch 5/5
1500/1500 [==============================] - 1s 815us/step - loss: 0.3209 - accuracy: 0.8855


In [27]:
model.evaluate(val_scaled, val_target) # 검증 세트 성능 확인

375/375 [==============================] - 0s 517us/step - loss: 0.3798 - accuracy: 0.8756


[0.3798390030860901, 0.8755833506584167]

## 옵티마이저
- 신경망의 가중치와 절편을 학습하기 위한 알고리즘 또는 방법
- 경사하강법(SGD, 네스테로프 모멘텀, RMSprop, Adam 등)

모멘텀 최적화와 RMSprop 의 장점을 접목한 Adam 을 사용하여 모델 훈련해보기
- 기본 RMSprop 보다 조금 나은 성능

In [28]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [29]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 1s 778us/step - loss: 0.5216 - accuracy: 0.8173
Epoch 2/5
1500/1500 [==============================] - 1s 755us/step - loss: 0.3908 - accuracy: 0.8597
Epoch 3/5
1500/1500 [==============================] - 1s 758us/step - loss: 0.3525 - accuracy: 0.8715
Epoch 4/5
1500/1500 [==============================] - 1s 734us/step - loss: 0.3255 - accuracy: 0.8829
Epoch 5/5
1500/1500 [==============================] - 1s 746us/step - loss: 0.3066 - accuracy: 0.8876


In [30]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 0s 528us/step - loss: 0.3425 - accuracy: 0.8772


[0.34253567457199097, 0.8771666884422302]